In [25]:
import pandas as pd
import numpy as np

data = pd.read_csv("archive/anime.csv")

In [26]:
data_after = data
data_after = data_after.drop(["scored_by", 'members','created_at', 'updated_at','background', 'main_picture','url', 'trailer_url', 'title_japanese'], axis=1)
selected_rows = data_after
selected_rows = selected_rows.loc[data_after['status'] == 'finished_airing']
selected_rows = selected_rows.loc[data_after['approved'] == True]


In [27]:
import datetime
from copy import deepcopy
def convert_to_seconds(time_str:str):
    if pd.isna(time_str):
        return time_str
    days = int(time_str.split(" ")[0])
    time_str = time_str[-8:]


    time_obj = datetime.datetime.strptime(time_str, r'%H:%M:%S')
    seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second
    seconds += days*24*60*60
    return seconds

def count_characters(text:str):
    if pd.isna(text):
        return 0
    return len(text)

def is_not_nan(field):
    return pd.isna(field)

In [28]:
converted_data =  deepcopy(selected_rows)
converted_data['episode_duration'] = converted_data['episode_duration'].apply(convert_to_seconds)
converted_data['total_duration'] = converted_data['total_duration'].apply(convert_to_seconds)

converted_data['synopsis_char_count'] = converted_data['synopsis'].apply(count_characters)
converted_data['title_char_count'] = converted_data['title'].apply(count_characters)
converted_data['start_date_day_of_year'] = pd.to_datetime(converted_data['start_date']).dt.dayofyear
converted_data['end_date_day_of_year'] = pd.to_datetime(converted_data['end_date']).dt.dayofyear
converted_data['broadcast_time'] = pd.to_timedelta(converted_data['broadcast_time']).dt.total_seconds() // 60


In [29]:
import ast
genres = converted_data['genres'].dropna()
genres = genres.apply(ast.literal_eval).explode().unique()
genres = list(genres)
genres.remove(np.nan)

def create_genres_vector(row):
    vector = [1 if word in row else 0 for word in genres]
    return ' '.join(map(str, vector))

In [30]:
themes = converted_data['themes'].dropna()
themes = themes.apply(ast.literal_eval).explode().unique()
themes = list(themes)
themes.remove(np.nan)

def create_themes_vector(row):
    vector = [1 if word in row else 0 for word in themes]
    return ' '.join(map(str, vector))

In [ ]:
demographics = converted_data['demographics'].dropna()
demographics = demographics.apply(ast.literal_eval).explode().unique()
demographics = list(demographics)
demographics.remove(np.nan)

def create_demographics_vector(row):
    vector = [1 if word in row else 0 for word in demographics]
    return ' '.join(map(str, vector))

In [31]:
ret = converted_data[['episodes', 'sfw', 'start_year','episode_duration','total_duration','synopsis_char_count','title_char_count','start_date_day_of_year','end_date_day_of_year','broadcast_time']]
ret = pd.concat([ret,converted_data[['type', 'source', 'rating', 'start_season', 'broadcast_day']]], axis=1)

ret = pd.concat([ret,converted_data['score']], axis=1)
ret = pd.concat([ret,converted_data[['genres','themes','demographics']]], axis=1)

ret['genres'] = ret['genres'].apply(create_genres_vector)
ret['themes'] = ret['themes'].apply(create_themes_vector)
ret['demographics'] = ret['demographics'].apply(create_demographics_vector)

ret.to_csv("data/anime_data_ludwig.csv")